In [102]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re


def load_data(data_path):
  data = pd.read_csv(data_path)  
  return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
    print(len(pallet))
trainentest = load_data("demand_kWtrain_val.csv")




1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
4925535


In [103]:
import math, datetime, time, random
# for data visualization
import matplotlib.pyplot as plt
#import missingno #doesn't work, why?!


In [104]:
### function to add time columns to a dataframe

def addtimecol(df, colname): ####input df and colname 
        df[colname] = pd.to_datetime(df[colname])         
        df['year'] = df[colname].dt.year
        df['month'] = df[colname].dt.month
        df['day'] = df[colname].dt.day
        df['hour'] = df[colname].dt.hour
        df['minute'] = df[colname].dt.minute        
        return df

In [105]:
###

In [235]:
#### feature: how many warm/new products are in? at time t

def f_warmproducts(time):                             ##from a single value in trainentest, get the amount of products that 
                                                      ###entered the building within 24 hours
    amount = 0    
    delta = timedelta(days = 1)    
    t = datetime.datetime.strptime(time, '%m/%d/%y %H:%M')
    newtime = delta + t
    inbound['load_start_datetime'] = pd.to_datetime(inbound['load_start_datetime'])  ###make column datetime objects
    
    for index, row in inbound.iterrows():                                      ### amount+1 for each order that entered the building within 24h
        if inbound.at[index, 'carrier_code'] != "CANCEL":
            if t <= inbound.at[index, 'load_start_datetime'] <= newtime :
                amount += measurewarmth(index, row)                            ### use measurewarmth to estimate the warmth that an entry brings in
                
    return amount
                


def measurewarmth(index, row):                                                ###currently, measure warmth uses the weight only to estimate the heat
    weight = inbound.at[index, 'net_weight']
    #medianwarmth(index,row) = 
    return weight




    

    




In [238]:
####using the functions I made

features = pd.DataFrame({'datetime_local': trainentest['datetime_local'][0:273987]})   ###making a dataframe for features

for index, rows in features.iterrows(): ###This takes extremely long
   
    features['f_warmproducts'] = f_warmproducts(features.at[index, 'datetime_local'])
    
print(features)
    


###using addtimecol to inbound given a certain column
#inbound = addtimecol(inbound, 'load_start_datetime')

    datetime_local  f_warmproducts
125    1/2/19 4:30       1411060.0
126    1/2/19 4:45       1411060.0
127    1/2/19 5:00       1411060.0
128    1/2/19 5:15       1411060.0
129    1/2/19 5:30       1411060.0


In [229]:
####estimating the current warmth of incoming products 
###

def medianwarmth(index,row):
    ###make new dataframe for this section 
    df = pd.DataFrame({'customer_code': inbound['customer_code'], 'carrier_code': inbound['carrier_code'], 'middle_temperature': inbound['middle_temperature']})
    ###filter through missing values/droplevel at missing values
    df = df.dropna()
    customer = inbound.at[index, 'customer_code']
    grouped = df.groupby('customer_code')['middle_temperature']
    temperatures = grouped.get_group(customer)["middle_temperature"].tolist()

    print(temperatures.median())
    


###find mean and variance in temperature per customer, plot them to see if usable.

#df['middle_temperature'].var() ### variance in total
#grouped.count()  ### amount per grouped category
#grouped.median() 
#grouped.var()



#ax = df.plot.bar(x='customer_code', y='middle_temperature', rot=0)
#for index,row in inbound.iterrows():
#    if inbound.at[index, 'middle_temperature']
#df.value_counts('customer_code')

In [152]:

#print(inbound["load_start_datetime"][1])

#print(inbound["load_start_datetime"].dt)
#datumwoord = pd.DataFrame({"date_string": inbound["load_start_datetime"][0:7]})
#s = datumwoord["date_string"]
#stime = pd.to_datetime(datumwoord["date_string"])
#datumwoord["date_string"] = pd.to_datetime(datumwoord["date_string"])
#print(datumwoord["date_string"].dt.year)
##id = ordersin.loc[ordersin['warehouse_order_number'] == 1290649.0, 'load_start_datetime'][0]
#stime = datetime.datetime.strptime(s[1],'%Y-%m-%d %H:%M:%S')
#print(stime)
